In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 

# Categorical Variables

# Numerical Variables

In [10]:
def filter_by_quarter(dataset, quarter):    
    filtered_dataset = dataset[dataset["Quarter"] == quarter]
    filtered_dataset.drop("Quarter", axis= 1, inplace= True)
    filtered_dataset.reset_index(drop= True, inplace= True)
    return filtered_dataset

## "Task Order" as distributor

In [11]:
# Select category as distributor
distributor = "Task Order"

# Define features
new_feature = "Total Shipped Quantity"
features = ["Average Days Late", "Illustrative Price", "Shipped Quantity", "Base Unit Multiplier", 
            "Order Cycle Time", "Latest Actual Delivery Date Fiscal Quarter Year"]
features.append(distributor)
features.append(new_feature)

# Create dataset
usaid = pd.read_csv("USAID_GHSC-PSM_Health_Commodity_Delivery_Dataset_20240105.csv", header= 0, low_memory= False)
usaid[new_feature] = usaid["Shipped Quantity"] * usaid["Base Unit Multiplier"]
usaid = usaid[features]
usaid[["Latest Actual Delivery Date Fiscal Quarter Year", "Quarter"]] = \
  usaid["Latest Actual Delivery Date Fiscal Quarter Year"].str.split("-", expand= True)
usaid.drop(["Base Unit Multiplier", "Shipped Quantity", "Latest Actual Delivery Date Fiscal Quarter Year"], axis= 1, inplace= True)

### For all possible timemframe ($k$ number of possible distributors)

In [12]:
dataset = usaid.copy()
dataset.drop(["Task Order", "Quarter"], axis = 1, inplace= True)
dataset.dropna(inplace= True)
dataset = dataset.sample(frac=1, random_state= 42).reset_index(drop=True)

n_distributors = [2,5,15,30,60]

for _, value in enumerate(n_distributors):
    chunk_size = dataset.shape[0] // value
    chunks = [dataset[i:i+chunk_size].copy().reset_index(drop= True) for i in range(0, dataset.shape[0], chunk_size)]    
    dataset_final = pd.concat(chunks, axis= 1)
    dataset_final.to_csv(f"to_wy_{value}distr.csv")